In [14]:
import numpy as np
import random
import html
import os
import pandas as pd
import requests
from bs4 import BeautifulSoup
import lxml
import rapidfuzz
import tqdm
import matplotlib as plt
import time
import json
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import glob
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 50)
pd.set_option('display.float_format', '{:.2f}'.format)

In [16]:
#Non enriched Dahua df (Baseline df)
df_dahua = pd.read_csv(r'C:\Users\LENOVO\Documents\GitHub\Enrichment_database\data\raw\dahua_df.csv')
df_dahua.head()

,Unnamed: 0,year,name,name_clean,name_canonical,description,slug,website,email,linkedin,twitter,instagram,facebook,employee_count,size,industry,type,country,founded,locality,latest_funding_raw,investors,total_funding,number_of_funding_rounds,business_model_type,latest_funding_amount,latest_funding_currency,source
0,2,2025,DAHUA TECHNOLOGY,NaN,NaN,Dahua Technology is a world-leading video-cent...,NaN,https://www.dahuatech.com,du_rui1@dahuatech.com,https://www.linkedin.com/company/dahua-technology,NaN,https://www.instagram.com/dahuaglobal,NaN,0,NaN,ENABLING TECHNOLOGIES,NaN,CN,0,"HANGZHOU, ZHEJIANG",NaN,NaN,0.00,0,NaN,0.00,NaN,SMART_CITY


In [17]:
#Manually enriched df
df_dahua_extracted = pd.read_csv(r'C:\Users\LENOVO\Documents\GitHub\Enrichment_database\data\raw\dahua-technology_overview_extracted.csv')
df_dahua_extracted.head()

,cb_rank,founded_date,legal_name,operating_status,stock_symbol,company_profit_type,founders,phone_number,contact_email,monthly_web_visits,web_traffic_growth,location_identifiers,location_city,location_region,location_country,description_short,company_type,last_funding_type,categories,last_funding_amount,last_funding_date,lead_investor,num_employees,activity_trend,key_people,about_company,investments_count,employee_profiles_count,investments_announced_date,investments_transaction_name,investments_number_of_investors,investments_money_raised,investments_lead_investors,investments_announced_dates,investments_announced_date1,investments_announced_date2,investments_announced_date3,investments_organization_names,investments_organization_name1,investments_organization_name2,investments_organization_name3,investments_money_raised1,investments_money_raised2,investments_money_raised3,news_dates,news_sources,news_source1,news_source2,news_source3,news_source4,news_headlines,news_headline1,news_headline2,news_headline3,news_headline4,news_links,news_link1,news_link2,news_link3,news_link4,number_of_funding_rounds,faqs,faqs_list,faqs1,faqs1_answer,faqs2,faqs2_answer,faqs3,faqs3_answer,faqs4,faqs4_answer,interest_signals_table,interest_signals_list,interest_signal1_topic,interest_signal1_score,interest_signal2_topic,interest_signal2_score,interest_signal3_topic,interest_signal3_score,actively_used_products_table,actively_used_products_amount,actively_used_products_list,actively_used_product1,actively_used_product2,actively_used_product3,active_website_tech_table,active_website_tech_count,active_website_tech_table_list,active_website_tech1,active_website_tech2,active_website_tech3,ip_table,ip_patents_count,ip_primar_patents_category,ip_trademarks_count,ip_trademarks_most_popular_class
0,24411,"Mar 12, 2001","Zhejiang Dahua Technology Co., Ltd",Active,SZSE:002236,For Profit,Fu Fiquan,8657187688883,overseas@dahuatech.com,455812,11.02,"Hangzhou, Zhejiang, China, Hangzhou, Zhejiang,...",Hangzhou,Zhejiang,China,Dahua Technology is a world-leading video surv...,For Profit,Post-IPO Equity,"Information Technology, Security, Video",CN¥5.1B,Mar 2021,China Mobile,10001+,Among all Video companies,"Zhang Xingming: Executive President, Wu Jian: ...",Dahua Technology is a world-leading video surv...,3,14,"Mar 29, 2021",Post-IPO Equity - Dahua Technology,1,"— , $150K , CN¥30M",China Mobile,"Apr 22, 2025, Feb 13, 2018, Aug 7, 2017","Apr 22, 2025","Feb 13, 2018","Aug 7, 2017","Renxin Technology, Adjoint, MAYCUR.COM",Renxin Technology,Adjoint,MAYCUR.COM,—,$150K,CN¥30M,NaN,"Boston Globe, Investing.com, Tech in Asia, Inv...",Boston Globe,Investing.com,Tech in Asia,Investing.com,US government allowed and even helped US firms...,US government allowed and even helped US firms...,US agency votes to tighten restrictions on Chi...,US agency tightens curbs on Chinese tech firms...,FCC moves to bar Hong Kong telecom carrier fro...,https://www.bostonglobe.com/2025/10/29/nation/...,https://www.bostonglobe.com/2025/10/29/nation/...,https://www.investing.com/news/stock-market-ne...,https://www.techinasia.com/news/agency-tighten...,https://www.investing.com/news/stock-market-ne...,1,Where is Dahua Technology's headquarters?Dahua...,"(""Where is Dahua Technology's headquarters?"", ...",Where is Dahua Technology's headquarters?,"Dahua Technology is located in Hangzhou, Zheji...",Who invested in Dahua Technology?,Dahua Technology is funded by China Mobile.,When was the last funding round for Dahua Tech...,Dahua Technology closed its last funding round...,Who are Dahua Technology's competitors?,Alternatives and possible competitors to Dahua...,| Topic | Surge Score | | --- | --- | | ![Comm...,"('Common Access Card (CAC)', '100'), ('Ericsso...",Common Access Card (CAC),100,Ericsson,99,Incident and Problem Management,98,4 Dahua Technology uses 4 technology products ...,4,"Microsoft Outlook, DigiCert, GlobalSign",Microsoft Outlook,DigiCert,GlobalSign,36 Dahua Technology is actively 

In [21]:
df_dahua_crunchbase = pd.read_csv(r'C:\Users\LENOVO\Documents\GitHub\Enrichment_database\data\Company_core_data\dataset_crunchbase-organzation-profile_2025-12-02_13-56-11-489.csv')
df_dahua_crunchbase.head()

,about,active_tech_count,address,apptopia/0/image,apptopia/0/monthly_downloads,apptopia/0/name,apptopia/0/stores/0,apptopia/1/image,apptopia/1/monthly_downloads,apptopia/1/name,apptopia/1/stores/0,apptopia/2/image,apptopia/2/monthly_downloads,apptopia/2/name,apptopia/2/stores/0,apptopia/3/image,apptopia/3/monthly_downloads,apptopia/3/name,apptopia/3/stores/0,apptopia/4/image,apptopia/4/monthly_downloads,apptopia/4/name,apptopia/4/stores/0,apptopia/5/image,apptopia/5/monthly_downloads,apptopia/5/name,apptopia/5/stores/0,apptopia/6/image,apptopia/6/monthly_downloads,apptopia/6/name,apptopia/6/stores/0,apptopia/7/image,apptopia/7/monthly_downloads,apptopia/7/name,apptopia/7/stores/0,apptopia/8/image,apptopia/8/monthly_downloads,apptopia/8/name,apptopia/8/stores/0,apptopia/9/image,apptopia/9/monthly_downloads,apptopia/9/name,apptopia/9/stores/0,apptopia_total_downloads,apptopia_total_downloads_mom_pct,bombora/0/category,bombora/0/score,bombora/0/topic,bombora/0/weeks_surging,bombora/0/wow_growth,bombora/1/category,bombora/1/score,bombora/1/topic,bombora/1/weeks_surging,bombora/1/wow_growth,bombora/2/category,bombora/2/score,bombora/2/topic,bombora/2/weeks_surging,bombora/2/wow_growth,bombora/3/category,bombora/3/score,bombora/3/topic,bombora/3/weeks_surging,bombora/3/wow_growth,bombora/4/category,bombora/4/score,bombora/4/topic,bombora/4/weeks_surging,bombora/4/wow_growth,bombora/5/category,bombora/5/score,bombora/5/topic,bombora/5/weeks_surging,bombora/5/wow_growth,bombora/6/category,bombora/6/score,bombora/6/topic,bombora/6/weeks_surging,bombora/6/wow_growth,bombora/7/category,bombora/7/score,bombora/7/topic,bombora/7/weeks_surging,bombora/7/wow_growth,bombora/8/category,bombora/8/score,bombora/8/topic,bombora/8/weeks_surging,bombora/8/wow_growth,bombora/9/category,bombora/9/score,bombora/9/topic,bombora/9/weeks_surging,bombora/9/wow_growth,bombora_last_updated,built_with_num_technologies_used,built_with_tech/0/name,built_with_tech/0/num_companies_using,built_with_tech/0/technology_category/0,built_with_tech/1/name,built_with_tech/1/num_companies_using,built_with_tech/1/technology_category/0,built_with_tech/2/name,built_with_tech/2/num_companies_using,built_with_tech/2/technology_category/0,built_with_tech/3/name,built_with_tech/3/num_companies_using,built_with_tech/3/technology_category/0,built_with_tech/4/name,built_with_tech/4/num_companies_using,built_with_tech/4/technology_category/0,built_with_tech/5/name,built_with_tech/5/num_companies_using,built_with_tech/5/technology_category/0,built_with_tech/6/name,built_with_tech/6/num_companies_using,built_with_tech/6/technology_category/0,built_with_tech/7/name,built_with_tech/7/num_companies_using,built_with_tech/7/technology_category/0,built_with_tech/8/name,built_with_tech/8/num_companies_using,built_with_tech/8/technology_category/0,built_with_tech/9/name,built_with_tech/9/num_companies_using,built_with_tech/9/technology_category/0,builtwith_num_technologies_used,builtwith_tech/0/name,builtwith_tech/0/num_companies_using,builtwith_tech/0/technology_category/0,builtwith_tech/1/name,builtwith_tech/1/num_companies_using,builtwith_tech/1/technology_category/0,builtwith_tech/2/name,builtwith_tech/2/num_companies_using,builtwith_tech/2/technology_category/0,builtwith_tech/3/name,builtwith_tech/3/num_companies_using,builtwith_tech/3/technology_category/0,builtwith_tech/4/name,builtwith_tech/4/num_companies_using,builtwith_tech/4/technology_category/0,builtwith_tech/5/name,builtwith_tech/5/num_companies_using,builtwith_tech/5/technology_category/0,builtwith_tech/6/name,builtwith_tech/6/num_companies_using,builtwith_tech/6/technology_category/0,builtwith_tech/7/name,builtwith_tech/7/num_companies_using,builtwith_tech/7/technology_category/0,builtwith_tech/8/name,builtwith_tech/8/num_companies_using,builtwith_tech/8/technology_category/0,builtwith_tech/9/name,builtwith_tech/9/num_companies_using,builtwith_tech/9/technology_category/0,cb_rank,company_activity_level,company_id,company_ind

In [ ]:
df_dahua_crunchbase

In [20]:
df_dahua_crunchbase.columns.to_list()

['about',
 'active_tech_count',
 'address',
 'apptopia/0/image',
 'apptopia/0/monthly_downloads',
 'apptopia/0/name',
 'apptopia/0/stores/0',
 'apptopia/1/image',
 'apptopia/1/monthly_downloads',
 'apptopia/1/name',
 'apptopia/1/stores/0',
 'apptopia/2/image',
 'apptopia/2/monthly_downloads',
 'apptopia/2/name',
 'apptopia/2/stores/0',
 'apptopia/3/image',
 'apptopia/3/monthly_downloads',
 'apptopia/3/name',
 'apptopia/3/stores/0',
 'apptopia/4/image',
 'apptopia/4/monthly_downloads',
 'apptopia/4/name',
 'apptopia/4/stores/0',
 'apptopia/5/image',
 'apptopia/5/monthly_downloads',
 'apptopia/5/name',
 'apptopia/5/stores/0',
 'apptopia/6/image',
 'apptopia/6/monthly_downloads',
 'apptopia/6/name',
 'apptopia/6/stores/0',
 'apptopia/7/image',
 'apptopia/7/monthly_downloads',
 'apptopia/7/name',
 'apptopia/7/stores/0',
 'apptopia/8/image',
 'apptopia/8/monthly_downloads',
 'apptopia/8/name',
 'apptopia/8/stores/0',
 'apptopia/9/image',
 'apptopia/9/monthly_downloads',
 'apptopia/9/name',


## Enrichment using Apify Actors

In [ ]:
#LinkedIn Scraper 1 
dahua_lk1 = pd.read_csv(r"C:\Users\jomap\Desktop\LatamConnect\Projects\Enrichment_databsase_building\Enrichment_database\data\Company_core_data\LK_scraper1\dataset_linkedin-company-scraper_2025-12-10_17-10-59-628.csv")

In [22]:
dahua_lk1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 94 columns):
 #   Column                                         Non-Null Count  Dtype  
---  ------                                         --------------  -----  
 0   data/affiliated_companies/0/company_id         1 non-null      int64  
 1   data/affiliated_companies/0/linkedin_url       1 non-null      object 
 2   data/affiliated_companies/0/name               1 non-null      object 
 3   data/affiliated_companies/1/company_id         1 non-null      int64  
 4   data/affiliated_companies/1/linkedin_url       1 non-null      object 
 5   data/affiliated_companies/1/name               1 non-null      object 
 6   data/affiliated_companies/2/company_id         1 non-null      int64  
 7   data/affiliated_companies/2/linkedin_url       1 non-null      object 
 8   data/affiliated_companies/2/name               1 non-null      object 
 9   data/affiliated_companies/3/company_id         1 non-null 

In [24]:
#Linkedin Scraper 3
dahua_lk3= pd.read_csv(r"C:\Users\LENOVO\Documents\GitHub\Enrichment_database\data\Company_core_data\LK_scraper3\dataset_linkedin-company-details-3.csv")
dahua_lk3.head()

,affiliated_companies/by_employees/0,affiliated_companies/by_employees/1,affiliated_companies/by_employees/2,affiliated_companies/by_employees/3,affiliated_companies/by_employees/4,affiliated_companies/by_jobs/0,affiliated_companies/by_jobs/1,affiliated_companies/by_jobs/2,affiliated_companies/by_jobs/3,affiliated_companies/by_jobs/4,affiliated_companies/showcases/0,affiliated_companies/showcases/1,affiliated_companies/showcases/2,affiliated_companies/showcases/3,basic_info/description,basic_info/founded_info/day,basic_info/founded_info/month,basic_info/founded_info/year,basic_info/industries/0,basic_info/is_verified,basic_info/linkedin_url,basic_info/name,basic_info/page_type,basic_info/specialties/0,basic_info/specialties/1,basic_info/specialties/2,basic_info/specialties/3,basic_info/specialties/4,basic_info/specialties/5,basic_info/specialties/6,basic_info/specialties/7,basic_info/specialties/8,basic_info/specialties/9,basic_info/specialties/10,basic_info/specialties/11,basic_info/specialties/12,basic_info/specialties/13,basic_info/specialties/14,basic_info/specialties/15,basic_info/universal_name,basic_info/verification/is_verified,basic_info/verification/last_verified_at,basic_info/website,call_to_action/text,call_to_action/type,call_to_action/url,call_to_action/visible,company_urn,corporate_relationships/acquirer_company,corporate_relationships/parent_company,funding/crunchbase_url,funding/latest_round/date,funding/latest_round/investors_count,funding/latest_round/type,funding/latest_round/url,funding/total_rounds,hashtags/0,hashtags/1,hashtags/2,input_identifier,links/crunchbase,links/job_search,links/linkedin,links/sales_navigator,links/website,locations/geo_coordinates/latitude,locations/geo_coordinates/longitude,locations/headquarters/city,locations/headquarters/country,locations/headquarters/description,locations/headquarters/is_hq,locations/headquarters/line1,locations/headquarters/line2,locations/headquarters/postal_code,locations/headquarters/state,locations/offices/0/city,locations/offices/0/country,locations/offices/0/description,locations/offices/0/is_hq,locations/offices/0/line1,locations/offices/0/line2,locations/offices/0/postal_code,locations/offices/0/region,locations/offices/0/state,locations/offices/1/city,locations/offices/1/country,locations/offices/1/description,locations/offices/1/is_hq,locations/offices/1/line1,locations/offices/1/line2,locations/offices/1/postal_code,locations/offices/1/region,locations/offices/1/state,locations/offices/2/city,locations/offices/2/country,locations/offices/2/description,locations/offices/2/is_hq,locations/offices/2/line1,locations/offices/2/line2,locations/offices/2/postal_code,locations/offices/2/region,locations/offices/2/state,locations/offices/3/city,locations/offices/3/country,locations/offices/3/description,locations/offices/3/is_hq,locations/offices/3/line1,locations/offices/3/line2,locations/offices/3/postal_code,locations/offices/3/region,locations/offices/3/state,locations/offices/4/city,locations/offices/4/country,locations/offices/4/description,locations/offices/4/is_hq,locations/offices/4/line1,locations/offices/4/line2,locations/offices/4/postal_code,locations/offices/4/region,locations/offices/4/state,locations/offices/5/city,locations/offices/5/country,locations/offices/5/description,locations/offices/5/is_hq,locations/offices/5/line1,locations/offices/5/line2,locations/offices/5/postal_code,locations/offices/5/region,locations/offices/5/state,media/cover_url,media/cropped_cover_url,media/logo_url,phone,similar_companies/0,similar_companies/1,similar_companies/2,similar_companies/3,similar_companies/4,similar_companies/5,similar_companies/6,similar_companies/7,similar_companies/8,similar_companies/9,similar_companies/10,similar_companies/11,stats/employee_count,stats/employee_count_range/end,stats/employee_count_range/start,stats/follower_count,stats/student_count,tagline
0,13342122,18491057,13341240,28166308,11199116,13342122,18491057,13341240,2816

In [25]:
#LinkedIn Scraper 4
dahua_lk4= pd.read_csv(r"C:\Users\LENOVO\Documents\GitHub\Enrichment_database\data\Company_core_data\LK_scraper4\dataset_company-linkedin-profile-4_2025-12-04_13-23-14-620.csv")
dahua_lk4.head()

,company_name,company_id,company_profile,company_website,company_address_type,company_street,company_locality,company_region,company_postal_code,company_country,company_employees_on_linkedin,company_followers_on_linkedin,company_logo,company_cover_image,company_slogan,company_twitter_description,company_about_us,company_industry,company_size,company_headquarters,company_organization_type,company_founded,company_specialties,input_url
0,Dahua Technology Co. LTD,2845844,https://cn.linkedin.com/company/dahua-technology,http://www.dahuasecurity.com/,PostalAddress,"No.1199, Bin'an Road, Binjiang District, Hangz...",NaN,NaN,NaN,NaN,5093,311313,https://media.licdn.com/dms/image/v2/C560BAQEs...,https://media.licdn.com/dms/image/v2/D563DAQEW...,NaN,"Dahua Technology Co. LTD | 311,313 followers o...","Zhejiang Dahua Technology Co., Ltd. (Dahua Tec...",Information Technology & Services,"10,001+ employees","Hangzhou, Zhejiang",Public Company,NaN,"DVRs/NVR/IVS, IP cameras, Security Solution Su...",https://www.linkedin.com/company/dahua-technol...


In [27]:
#LinkedIn Scraper 5 
dahua_lk5= pd.read_csv(r"C:\Users\LENOVO\Documents\GitHub\Enrichment_database\data\Company_core_data\LK_scraper5\dataset_linkedin-company-profile-scraper_5.csv")
dahua_lk5.head()

,affiliated_pages_data,background_cover_image_url,company_id,company_name,company_type,description,employees/0/image_url,employees/0/name,employees/0/profile_url,employees/0/role,employees/1/image_url,employees/1/name,employees/1/profile_url,employees/1/role,employees/2/image_url,employees/2/name,employees/2/profile_url,employees/2/role,employees/3/image_url,employees/3/name,employees/3/profile_url,employees/3/role,employees_data,followers,founded_year,headcount,hq_address/address,hq_address/city_state,hq_address/is_primary,hq_address/map_url,hq_address/street_address,industry,location,profile_pic_url,similar_pages_data,specialities,tagline,universal_name_id,update_data/0/comments,update_data/0/content,update_data/0/media/0/alt,update_data/0/media/0/url,update_data/0/media/1/alt,update_data/0/media/1/url,update_data/0/media/2/alt,update_data/0/media/2/url,update_data/0/media/3/alt,update_data/0/media/3/url,update_data/0/media/4/alt,update_data/0/media/4/url,update_data/0/posted_time,update_data/0/reaction_types/0,update_data/0/reaction_types/1,update_data/0/reaction_types/2,update_data/0/reactions,update_data/1/comments,update_data/1/content,update_data/1/media/0/alt,update_data/1/media/0/url,update_data/1/media/1/alt,update_data/1/media/1/url,update_data/1/media/2/alt,update_data/1/media/2/url,update_data/1/media/3/alt,update_data/1/media/3/url,update_data/1/media/4/alt,update_data/1/media/4/url,update_data/1/posted_time,update_data/1/reaction_types/0,update_data/1/reaction_types/1,update_data/1/reaction_types/2,update_data/1/reactions,update_data/2/comments,update_data/2/content,update_data/2/media/0/alt,update_data/2/media/0/url,update_data/2/media/1/alt,update_data/2/media/1/url,update_data/2/media/2/alt,update_data/2/media/2/url,update_data/2/media/3/alt,update_data/2/media/3/url,update_data/2/media/4/alt,update_data/2/media/4/url,update_data/2/posted_time,update_data/2/reaction_types/0,update_data/2/reaction_types/1,update_data/2/reaction_types/2,update_data/2/reactions,update_data/3/comments,update_data/3/content,update_data/3/media/0/alt,update_data/3/media/0/url,update_data/3/media/1/alt,update_data/3/media/1/url,update_data/3/media/2/alt,update_data/3/media/2/url,update_data/3/media/3/alt,update_data/3/media/3/url,update_data/3/media/4/alt,update_data/3/media/4/url,update_data/3/posted_time,update_data/3/reaction_types/0,update_data/3/reaction_types/1,update_data/3/reaction_types/2,update_data/3/reactions,update_data/4/comments,update_data/4/content,update_data/4/media/0/alt,update_data/4/media/0/url,update_data/4/media/1/alt,update_data/4/media/1/url,update_data/4/media/2/alt,update_data/4/media/2/url,update_data/4/media/3/alt,update_data/4/media/3/url,update_data/4/media/4/alt,update_data/4/media/4/url,update_data/4/posted_time,update_data/4/reaction_types/0,update_data/4/reaction_types/1,update_data/4/reaction_types/2,update_data/4/reactions,update_data/5/comments,update_data/5/content,update_data/5/media/0/alt,update_data/5/media/0/url,update_data/5/media/1/alt,update_data/5/media/1/url,update_data/5/media/2/alt,update_data/5/media/2/url,update_data/5/media/3/alt,update_data/5/media/3/url,update_data/5/media/4/alt,update_data/5/media/4/url,update_data/5/posted_time,update_data/5/reaction_types/0,update_data/5/reaction_types/1,update_data/5/reaction_types/2,update_data/5/reactions,update_data/6/comments,update_data/6/content,update_data/6/media/0/alt,update_data/6/media/0/url,update_data/6/media/1/alt,update_data/6/media/1/url,update_data/6/media/2/alt,update_data/6/media/2/url,update_data/6/media/3/alt,update_data/6/media/3/url,update_data/6/media/4/alt,update_data/6/media/4/url,update_data/6/posted_time,update_data/6/reaction_types/0,update_data/6/reaction_types/1,update_data/6/reaction_types/2,update_data/6/reactions,update_data/7/comments,update_data/7/content,update_data/7/media/0/alt,update_data/7/media/0/url,update_data/7/media/1/alt,update_data/7/media/1/url,update_data/7/media/2/alt,update_data/7/media/2/